In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os

if not getattr(sys, "_src_path_added", False):
    src_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
    sys._src_path_added = True 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from MTGA.MTGA import *

from MTGA.MTGAInstance import Instance
import MTGA.Generators as Generators
import MTGA.Mutators as Mutators
import MTGA.WeightGenerators as WeightGenerators
import Fixers as Fixers
import Evaluators as Evaluators

from MTGA.GeneVisualizer import GeneEvolutionRenderer

from ProblemDef import FirefighterProblem
from Utils import *


## Main Instance

In [ ]:
instance = Instance("problems/p1.json",
    evaluator=Evaluators.CummulativeEvaluator(),
    mutator=Mutators.WalkMutator(),
    generator=Generators.Generator(),
    fixer=Fixers.ChoiceFixer(),
    weight_generator=WeightGenerators.SoftmaxWeights())

In [ ]:
data = optimize_and_collect(instance, number_of_iterations=20)

In [ ]:
vis = GeneEvolutionRenderer(independent_ylim=False)
vis.visualize(data)
# vis.visualize(data, "output/p1_evolution_softmax_lim1.gif")

In [ ]:
instance.solutions

In [ ]:
def check_range(r, correction_function):
    output = []
    for steepness in r:
        config = {
            'correction_function': correction_function,
            'correction_sigmoid_steepness': steepness,
            'number_of_iterations': 5
        }
            
        
        instance = Instance("problems/p1.json",
            evaluator=Evaluators.CummulativeEvaluator(),
            mutator=Mutators.WalkMutator(),
            generator=Generators.Generator(),
            fixer=Fixers.ChoiceFixer(),
            weight_generator=WeightGenerators.SoftmaxWeights())

        output_name = f"output/p1_evolution_testing_steepness.{steepness}.gif"

        data = optimize_and_collect(instance, **config)
        vis = GeneEvolutionRenderer(independent_ylim=False)
        output.append(vis.visualize(data))

In [ ]:
# output = check_range([16, 32, 64, 128, 256], "sigmoid")

In [ ]:
def render(steepness):
    instance = Instance("problems/p1.json",
        evaluator=Evaluators.CummulativeEvaluator(),
        mutator=Mutators.WalkMutator(),
        generator=Generators.Generator(),
        fixer=Fixers.ChoiceFixer(),
        weight_generator=WeightGenerators.SoftmaxWeights())
    
    data = optimize_and_collect(instance, number_of_iterations=5, print_updates=False, correction_function='smooth_clip')
    vis = GeneEvolutionRenderer(independent_ylim=False)
    return vis.visualize(data)

In [ ]:
# midpoint=mean, small steepness
# render(0.001)

In [ ]:
Utils.smooth_clip(4.1)

In [ ]:
mtga = MTGA(instance)
current_genes = mtga.initialize_genes()
mtga.generate_population(current_genes)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define the edges layer by layer
edges = [
    # Bottom layer
    (0, 1), (1, 2),
    # Connections between bottom and second layers
    (0, 3), (0, 4), (1, 4), (1, 5), (2, 5), (2, 6),
    # Second layer
    (3, 4), (4, 5), (5, 6),
    # Connections between second and third layers
    (3, 7), (3, 8), (4, 8), (4, 9), (5, 9), (5, 10), (6, 10), (6, 11),
    # Third layer
    (7, 8), (8, 9), (9, 10), (10, 11),
    # Connections between third and fourth layers
    (7, 12), (9, 14), (11, 16), 
    # Fourth layer
    (12, 13), (13, 14), (14, 15), (15, 16),
    # Connections between fourth and fifth layers
    (12, 17), (13, 17), (13, 18), (14, 18), (14, 19), (15, 19), (15, 20), (16, 20),
    # Fifth layer
    (17, 18), (18, 19), (19, 20),
    # Connections between fifth and top layer
    (17, 21), (18, 21), (18, 22), (19, 22), (19, 23), (20, 23),
    # Top layer
    (21, 22), (22, 23),
]

# Create the undirected graph
G = nx.Graph()
G.add_edges_from(edges)

# Draw the graph
# pos = nx.spring_layout(G)  # Use spring layout for better visualization
nx.draw(G, pos, with_labels=True, node_size=700, node_color="lightblue", font_size=10, font_weight="bold", edge_color="gray")
plt.title("Graph from Image")
plt.show()


In [6]:
from Evaluators import IncrementalCummulativeEvaluator
import numpy as np

perm = np.random.permutation(10)
print(perm)

e = IncrementalCummulativeEvaluator()
e.pick_k(perm, 4, set([perm[1], perm[3]]))


[6 7 5 3 8 9 0 1 4 2]


array([6, 5, 8, 9])